In [19]:
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, date
from meteostat import Stations, Daily, Hourly
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from keras.models import Sequential
from keras.layers import Dense, Activation
import requests
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

ModuleNotFoundError: No module named 'tensorflow'

In [3]:
# Retrieve Strava activity data as JSON response
auth_url = "https://www.strava.com/oauth/token"
activites_url = "https://www.strava.com/api/v3/athlete/activities"

payload = {
    'client_id': "114186",
    'client_secret': '535f4e9fce8216e83d23ecf9644b9b3c00f05ba5',
    'refresh_token': '3f8ab023c1f397d9fa98ed15ca94977267deb54c',
    'grant_type': "refresh_token",
    'f': 'json'
}

# First API call to request access token
# Access tokens expire, so this saves you manual work of regenerating access tokens manually
print("Requesting Token...\n")
res = requests.post(auth_url, data=payload, verify=False)
access_token = res.json()['access_token']
print("Access Token = {}\n".format(access_token))

# Make API request to retrieve the last 400 activities
header = {'Authorization': 'Bearer ' + access_token}
param1 = {'per_page': 200, 'page': 1}
param2 = {'per_page': 200, 'page': 2}
param3 = {'per_page': 200, 'page': 3}
param4 = {'per_page': 200, 'page': 4}
my_dataset1 = requests.get(activites_url, headers=header, params=param1).json()
my_dataset2 = requests.get(activites_url, headers=header, params=param2).json()
my_dataset3 = requests.get(activites_url, headers=header, params=param3).json()
my_dataset4 = requests.get(activites_url, headers=header, params=param4).json()

print(my_dataset1[0]["name"])
print(my_dataset1[0]["map"]["summary_polyline"])

print(my_dataset2[0]["name"])
print(my_dataset2[0]["map"]["summary_polyline"])

print(my_dataset3[0]["name"])
print(my_dataset3[0]["map"]["summary_polyline"])

print(my_dataset4[0]["name"])
print(my_dataset4[0]["map"]["summary_polyline"])

Requesting Token...

Access Token = 0312cf0fd31e8312dfaced85ae0cea11b050bbbe

Evening Run
iz{`HkuhsBLNBLF|ANTDFXHn@^j@LHLnAl@^VFLCDVn@d@~@@Nl@b@\t@J|@Eb@Wr@Gv@@TZZ`@f@DVJHARSl@^l@Pp@@XPLb@@p@c@d@JHCJKNCRYd@a@FCX?v@QZBb@GZk@FGVMVY^Q~@MV_@@]Oc@A}@JBI?AHr@pCd@xARv@Db@ZnAvBpHv@tDNp@Vp@D\@v@C`@ODEE?KHYLIbA_BPw@Ge@[{@HKvAwAp@}@r@u@`@m@\_@ZWX_@r@g@|@kAr@q@b@Wl@{@|@}@r@i@LWZWVY`@Yt@_A\YPW`@[p@y@\[b@s@j@m@Vc@^c@n@}@pAwAXUn@_AlA}AXg@fCiCl@{@`@[RY|@_AVi@NQ^o@VQ|@mAXMXETOp@gAh@c@z@gA^[V]pAoAJWdAyAzByBDORYf@e@v@oAtA_Av@c@h@Wp@S^Wt@W~@g@bAWJGfB]^MJOt@Mt@]`A]pAYj@E`@Mf@Iz@Ij@Qf@HJVTXB@rCKpCWRFHLBLBt@?VK|AC|D@FPZLBJCRYN{@ZmHDmCXk[FmHEw@KVUZ@r@BTCHKFoBIy@cCc@gBWq@Q[IA_@BsATYj@q@j@i@XQLkA^gAr@]L[RW@YTk@r@qAt@c@R[FWIu@`AW?MNQLSB{@ZQ?UgAQGy@LgAj@_@BMIc@ES@QH_@Z}@ZUVAD[T}Al@k@ZYFa@@c@POJy@?c@xAMp@O|AWj@]^a@l@eA~@QNK?_@\]d@uAxAeBzBk@bA}AhA_@f@u@dA{@~A[^YFOQMEU_@[wAE_AIE_AXU@eAt@[^YTm@ZO^a@Z]HOf@GHcBj@WNYFiAb@M@_@Re@Jg@Po@@iAl@aAZUNMRa@Fi@Xo@Oe@QIa@Ym@MIIWo@cAIIQIW?QDI@OJoAAi@Bm@DkBZc@?k@TcAXi@b@kAb@c@\iBh@]

In [4]:
# Flatten nested JSON response into a Dataframe
activities1 = pd.json_normalize(my_dataset1)
activities2 = pd.json_normalize(my_dataset2)
activities3 = pd.json_normalize(my_dataset3)
activities4 = pd.json_normalize(my_dataset4)

# Combine into one dataframe
activities = pd.concat([activities1, activities2, activities3, activities4])
activities = activities.reset_index()
activities = activities.drop(columns=["index"])

print("Dataframe shape: ", activities.shape, "\n")
print("Dataframe columns: \n", activities.columns)

Dataframe shape:  (800, 55) 

Dataframe columns: 
 Index(['resource_state', 'name', 'distance', 'moving_time', 'elapsed_time',
       'total_elevation_gain', 'type', 'sport_type', 'workout_type', 'id',
       'start_date', 'start_date_local', 'timezone', 'utc_offset',
       'location_city', 'location_state', 'location_country',
       'achievement_count', 'kudos_count', 'comment_count', 'athlete_count',
       'photo_count', 'trainer', 'commute', 'manual', 'private', 'visibility',
       'flagged', 'gear_id', 'start_latlng', 'end_latlng', 'average_speed',
       'max_speed', 'average_cadence', 'has_heartrate', 'average_heartrate',
       'max_heartrate', 'heartrate_opt_out', 'display_hide_heartrate_option',
       'elev_high', 'elev_low', 'upload_id', 'upload_id_str', 'external_id',
       'from_accepted_tag', 'pr_count', 'total_photo_count', 'has_kudoed',
       'suffer_score', 'athlete.id', 'athlete.resource_state', 'map.id',
       'map.summary_polyline', 'map.resource_state', 'ave

In [5]:
# Flag race activities
# only keep run activties
running_activities = activities.loc[activities["type"] == "Run"]
running_activities = running_activities.drop(columns=["type"])
running_activities["race"] = False

# manually mark race activities as "race"=True
# Races: 11-11-2023 , 2-19-2023, 1-22-2023, 2-27-2022, 1-23-2022, 5-6-2023, 11-24-2022, 12-4-2022
# Relays: 3-28-2021, 3-27-2021, 3-27-2021, 3-26-2023, 3-25-2023, 3-25-2023, 8-26-2023, 8-25-2023, 8-25-2023
# NOT in current data pull -  1-23-2022, 2-27-2022, 3-28-2021, 3-27-2021, 3-27-2021
race_activity_ids = [10270133740,10201538669,8589932304,8434045492,9022328776,8162552673,8207198428]
relay_activity_ids = [8779880223,8777939900,8776825449,9721567283,9719999180,9725419043]

# uncomment to locate race activities and retrieve ids
#display(running_activities.loc[(pd.to_datetime(running_activities["start_date_local"])).dt.date == date(2023,11,23)])
#display(running_activities.loc[running_activities.id == 10201538669])

for id in race_activity_ids:
        running_activities.loc[running_activities.id == id, "race"] = True

'''
for id in relay_activity_ids:
        running_activities.loc[running_activities.id == id, "race"] = True
'''           

race_activities = running_activities.loc[running_activities["race"] == True]
display(race_activities)

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,workout_type,id,start_date,...,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp,race
1,2,Seattle Turkey Trot 15k 🦃,15251.8,3903,3903,36.3,Run,0.0,10270133740,2023-11-23T17:31:57Z,...,2,False,226.0,42128847,1,a10270133740,ss_bHjjciVcBcAgCq@mETsBxAuFjE]^e@bAAl@XTf@JjAF...,2,NaN,True
9,2,Richmond Marathon 2023,42360.6,10966,11140,198.3,Run,0.0,10201538669,2023-11-11T12:00:10Z,...,5,False,508.0,42128847,1,a10201538669,w_ddFretwMwBnEqChFmCvEq@~AwInQqDpGmCbGaFhJ}LzV...,2,NaN,True
217,2,Yosemite Half Marathon (see race highlights be...,20935.9,5372,5510,167.9,Run,0.0,9022328776,2023-05-06T13:01:35Z,...,1,False,194.0,42128847,1,a9022328776,yaxbFd_vxUFfBS|BaCvKGn@TvBAr@a@vCe@j@{BFm@l@Nt...,2,NaN,True
298,2,Austin Half Marathon,21294.7,5384,5384,184.4,Run,0.0,8589932304,2023-02-19T13:04:04Z,...,3,False,224.0,42128847,1,a8589932304,aavwDbtqsQbAH\Pj@Dt@`@vAh@Z\tH`Br@TtG~AnDrAj@B...,2,NaN,True
333,2,3M Half Marathon,21267.1,5192,5192,52.7,Run,0.0,8434045492,2023-01-22T13:31:17Z,...,2,False,237.0,42128847,1,a8434045492,gfnxD~qpsQoAbASBo@@o@ZgATgBF{CUuA?o@JyABm@NcAH...,2,NaN,True
377,2,SA Rock ‘n Roll,21294.8,5347,5394,83.7,Run,0.0,8207198428,2022-12-04T13:04:09Z,...,4,False,187.0,42128847,1,a8207198428,q`rrDffdxQmAAs@Ok@EeBJq@MkCCGA[]Fo@AUMi@GeA@k@...,2,NaN,True
386,2,Turkey trot - race edition,5113.7,1158,1158,12.0,Run,0.0,8162552673,2022-11-24T14:31:51Z,...,1,False,27.0,42128847,1,a8162552673,g`agE`dxmQ@_@Ck@]cCI_@Uc@m@o@o@i@k@o@e@yAKg@AQ...,2,NaN,True


In [6]:
# Helper functions
def ft_to_meters(dist_ft):
    return dist_ft * 0.3048

def miles_to_meters(dist_miles):
    return dist_miles * 1609.34

def z_score(df, col_name):
    df2 = df.copy()
    df2[col_name] = ( (df[col_name] - df[col_name].mean()) / df[col_name].std() )
    return df2

def normalize(df, col_name):
    df2 = df.copy()
    df2[col_name] = ( (df2[col_name] - df2[col_name].min()) / (df2[col_name].max() - df2[col_name].min()) )
    return df2

def seconds_to_timedelta(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)

    return str("%d:%02d:%02d" % (hours, minutes, seconds))

def min_per_mile(seconds, pace):
    
    return
    
    
# Check distribution of numeric columns to decide proper encoding method
def plot_elev_high(df):
    df["elev_high"].plot(kind="hist", title="Elevation High Distribution")
    print("elev_high mean:",df["elev_high"].mean(),"\nelev_high std distribution:",df["elev_high"].std())
    
def plot_avg_temp(df):
    df["avg_temp"].plot(kind="hist", title="Average Temperature Distribution")
    print("avg_temp mean:",df["avg_temp"].mean(),"\navg_temp std distribution:",df["avg_temp"].std())
    
def plot_avg_cadence(df):
    df["average_cadence"].plot(kind="hist", title="Average Cadence Distribution")
    print("average_cadence mean:",df["average_cadence"].mean(),"\naverage_cadence std distribution:",df["average_cadence"].std())

def plot_max_hr(df):
    df["max_heartrate"].plot(kind="hist", title="Maximum Heartrate Distribution")
    print("max_heartrate mean:",df["max_heartrate"].mean(),"\nmax_heartrate std distribution:",df["max_heartrate"].std())

def plot_avg_hr(df):   
    df["average_heartrate"].plot(kind="hist", title="Average Heartrate Distribution")
    print("average_heartrate mean:",df["average_heartrate"].mean(),"\naverage_heartrate std distribution:",df["average_heartrate"].std())

def plot_max_speed(df):
    df["max_speed"].plot(kind="hist", title="Maxiumum Speed Distribution")
    print("max_speed mean:",df["max_speed"].mean(),"\nmax_speed std distribution:",df["max_speed"].std())

def plot_elev_gain(df):
    df["total_elevation_gain"].plot(kind="hist", title="Total Elevation Gain Distribution")
    print("total_elevation_gain mean:",df["total_elevation_gain"].mean(),"\ntotal_elevation_gain std distribution:",df["total_elevation_gain"].std())

In [7]:
def handle_nulls(df, method):
    if method == "interpolation":
        df = df.interpolate()
        return df

def handle_nulls_race(input_df, training_df):
    race_activities = training_df.loc[training_df["race"] == True]
    print("race activities to get avgs")
    display(input_df)
    
    

In [8]:
# Retrieves weather data from a given set of locations (loc_df) across a date range (min_date -> max_date)
# Returns input timestamped dataframe (df) with an additional column, avg_temp
def get_weather(min_date, max_date, loc_df, df):

    # Get unique list of weather stations from locations
    print("Retrieving list of weather stations...")
    weather_stations = pd.DataFrame()
    loc_dict = {}
    for location in loc_df:
        stations = Stations()
        nearby_station = stations.nearby(location[0], location[1]).fetch(1)
        loc_dict[("[ "+str(location[0])+" "+str(location[1])+"]")] = nearby_station.index[0]
        weather_stations = pd.concat([weather_stations, nearby_station])

    weather_stations = weather_stations.drop_duplicates(subset='name')
    print("Weather stations retrieved.")
    display(weather_stations.head())
    
    # Retrieving Weather Data from Nearby Stations
    # Create dictionary to hold dataframes of weather data. Dict key is weather station id.
    print("Retrieving weather data...")
    weather_df_dict = {}
    for coord, station_id in loc_dict.items():
        weather_df_dict[station_id] = pd.DataFrame()
    
    # Retrieve at least a year of weather data
    today = date.today()
    last_year = today.year - 1
    a_year_ago_today = date(last_year, today.month, today.day)
    if a_year_ago_today < min_date:
        min_date = a_year_ago_today

    for idx, station in weather_stations.iterrows():
        # Daily Weather
        weather_df_dict[idx] = Daily(idx, start=datetime.combine(min_date, datetime.min.time()), end=datetime.combine(max_date, datetime.min.time())).fetch()

        # Hourly Weather
        #weather_df_dict[idx] = Hourly(idx, start=datetime.combine(min_date, datetime.min.time()), end=datetime.combine(max_date, datetime.min.time())).fetch()
    print("Weather data retrieved.")
    
    # Add weather data to loc_df
    print("Adding weather data to df...")
    
    # Add weather data to run_activities dataframe
    df["weather_station"] = None
    df["avg_temp"] = None

    for idx, activity in df.iterrows():
        dict_key = "[ " + str(activity['start_lat']) + " " + str(activity['start_lng']) + "]"
        if dict_key in loc_dict:
            weather_df = weather_df_dict[loc_dict[dict_key]]
            # check if date is in the future
            if today < activity["start_date_local"]:
                # if date is in future, use weather from last year
                new_date = date(activity["start_date_local"].year - 1, activity["start_date_local"].month, activity["start_date_local"].day)
                try:
                    df.at[idx,"weather_station"] = loc_dict[dict_key]
                    df.at[idx,"avg_temp"] = weather_df.loc[str(new_date)]["tavg"]
                except:
                    # weather not found for date / station
                    df.at[idx,"weather_station"] = loc_dict[dict_key]
                    df.at[idx,"avg_temp"] = np.nan
            try:
                df.at[idx,"weather_station"] = loc_dict[dict_key]
                df.at[idx,"avg_temp"] = weather_df.loc[str(activity["start_date_local"])]["tavg"]
            except:
                # weather not found for date / station
                df.at[idx,"weather_station"] = loc_dict[dict_key]
                df.at[idx,"avg_temp"] = np.nan
    
    print("Weather data added.")
    df = df.drop(labels=["weather_station"], axis=1)
    display(df.head())
                
    return df

In [9]:
# Returns df with encoded categorical and numeric columns.
def data_encoding(df, skip_encoding):
    df_encoded = df
    
    # encode time cyclically
    # split date time into seperate columns
    df_encoded.loc[:,"date-time"] = pd.to_datetime(df_encoded.start_date_local.astype(str)+ " " + df_encoded.start_time.astype(str ))
    df_encoded["year"] = df_encoded["date-time"].dt.year
    df_encoded["month"] = df_encoded["date-time"].dt.month
    df_encoded["day"] = df_encoded["date-time"].dt.day
    df_encoded["hour"] = df_encoded["date-time"].dt.hour
    df_encoded["min"] = df_encoded["date-time"].dt.minute
    df_encoded["sec"] = df_encoded["date-time"].dt.second
    df_encoded["total_seconds"] = df_encoded["hour"]*60*60 +  df_encoded["min"]*60 +  df_encoded["sec"]
    
    # cyclically encode date and time with sin / cos functions
    seconds_in_day = 24*60*60

    df_encoded["month_sin"] = np.sin(2*np.pi*df_encoded.month/12)
    df_encoded["month_cos"] = np.cos(2*np.pi*df_encoded.month/12)

    df_encoded["day_sin"] = np.sin(2*np.pi*df_encoded.day/31)
    df_encoded["day_cos"] = np.cos(2*np.pi*df_encoded.day/31)

    df_encoded["sec_sin"] = np.sin(2*np.pi*df_encoded.total_seconds/seconds_in_day)
    df_encoded["sec_cos"] = np.cos(2*np.pi*df_encoded.total_seconds/seconds_in_day)

    #df_encoded.plot.scatter("sec_sin", "sec_cos", title="Time of Day Encoded").set_aspect("equal")
    #df_encoded.plot.scatter("day_sin", "day_cos", title="Day of Month Encoded").set_aspect("equal")
    #df_encoded.plot.scatter("month_sin", "month_cos", title="Month of Year Encoded").set_aspect("equal")
    
    # one-hot encode year
    one_hot_year = pd.get_dummies(df_encoded["year"])
    df_encoded = df_encoded.join(one_hot_year)
    
    if not skip_encoding:
        print("HERE")
        # normalize or standardize numeric columns based on distribution of values (normal or not)
        df_encoded = z_score(df_encoded, "average_cadence")
        df_encoded = z_score(df_encoded, "average_heartrate")
        df_encoded = z_score(df_encoded, "max_heartrate")
        df_encoded = normalize(df_encoded, "total_elevation_gain")
        df_encoded = normalize(df_encoded, "elev_high")
        df_encoded = z_score(df_encoded, "max_speed")
        df_encoded = z_score(df_encoded, "avg_temp")
    
    # drop uncoded columns
    df_encoded = df_encoded.drop(columns=["date-time","year","month","day","hour","min","sec","total_seconds","start_time","start_date_local"])
    
    return df_encoded

In [10]:
def data_preprocessing(df, skip_encoding=False):
    
    # modify date/time/location columns
    df['start_date_local'] = pd.to_datetime(df['start_date_local'])
    df['start_time'] = df['start_date_local'].dt.time
    df['start_date_local'] = df['start_date_local'].dt.date
    df['start_lat'] = round(df['start_latlng'].str[0], 2)
    df['start_lng'] = round(df['start_latlng'].str[1], 2)
    df = df.drop(columns=['start_latlng'])
    
    # add weather data
    # if date is in future (i.e. for race prediction), get the weather from that location last year
    min_date, max_date = df['start_date_local'].min(), df['start_date_local'].max()
    locations = df.loc[:,['start_lat', 'start_lng']].drop_duplicates().values
    
    df = handle_nulls(df, "interpolation")
    df = get_weather(min_date, max_date, locations, df)
    df["avg_temp"].fillna((df["avg_temp"].mean()), inplace = True) # handle any null values in new avg_temp col
    df = df.drop(columns=['start_lat','start_lng'])
    print("How many nulls?") # verify there are no null values
    print(df.isnull().sum())
    df = data_encoding(df, skip_encoding)
    
    return df

In [16]:
# Model predicts moving time / average pace during a race based on race's location, elevation, and distance.
def race_predictor(model_training_df, input_param_dict, model_name):
    # Verify format of input parameters - race_date, start_latlng, distance, elev_gain, elev_high
    if len(input_param_dict) < 5:
        print("Missing input parameters. race_predictor is expecting a dictionary with keys: race_date, start_latlng, distance, elev_gain, elev_high.")
        return
    
    cols = ['start_date_local', 'start_latlng', 'distance', 'moving_time',  'total_elevation_gain', 'max_speed',
            'average_cadence', 'average_heartrate', 'max_heartrate', 'elev_high', 'race']
    model_training_df = model_training_df[cols]
    race_activities_df = model_training_df.loc[model_training_df["race"] == True]
    
    # populate input_df with avg values from other race activities
    input_df = pd.DataFrame( {
        "start_date_local": [input_param_dict["race_date"]],
        "start_latlng": [input_param_dict["start_latlng"]],
        "distance":  [miles_to_meters(input_param_dict["distance_miles"])],
        "moving_time": [None],
        "total_elevation_gain": [ft_to_meters(input_param_dict["elev_gain_ft"])],
        "max_speed": [race_activities_df["max_speed"].mean()],
        "average_cadence": [race_activities_df["average_cadence"].mean()],
        "average_heartrate": [race_activities_df["average_heartrate"].mean()],
        "max_heartrate": [race_activities_df["max_heartrate"].mean()],
        "elev_high": [ft_to_meters(input_param_dict["elev_high_ft"])],
        "race": True,
        "prediction": True}
                            
        )
    model_training_df["prediction"] = False
    display(model_training_df)
    model_df = pd.concat([model_training_df, input_df])
    display(model_df)
    training_data = data_preprocessing(model_df)
    
    print("TRAIONGON DTATA")
    display(training_data.loc[training_data["race"] == True])
    #race_activities = training_data.loc[training_data["race"] == True]
    
    #input_df["max_speed"] = race_activities["max_speed"].mean()
    #input_df["average_cadence"] = race_activities["average_cadence"].mean()
    #input_df["average_heartrate"] = race_activities["average_heartrate"].mean()
    #input_df["max_heartrate"] =  race_activities["max_heartrate"].mean()
    
    #print("calling data_preprocessing for input_df")
    #display(input_df.head())
    #race_input = data_preprocessing(input_df, skip_encoding="True")
    #display(race_input.head())
    
    if model_name == "linear_reg":
        x_race = training_data.loc[training_data["prediction"] == True]
        x_train = training_data.loc[training_data["prediction"] == False]
        
        #x = np.array(x_train.drop(columns=["moving_time"]))
        x = np.array(x_train.drop(columns=["moving_time","month_sin","month_cos","day_sin","day_cos"]))
        y = np.array(x_train["moving_time"])
        
        # Build Multiple Linear Regression model with scikit-learn
        model = LinearRegression().fit(x,y)
        
        # Print model results
        r_sq = model.score(x,y)
        print(f"coefficient of determination: {r_sq}")
        print(f"intercept: {model.intercept_}")
        print(f"coefficients: {model.coef_}")
        
        # Use model to predict moving_time for race
        #x_pred = np.array(x_race.drop(columns=["moving_time"]))
        x_pred = np.array(x_race.drop(columns=["moving_time","month_sin","month_cos","day_sin","day_cos"]))
    
        y_pred = model.predict(x_pred)
        
        print("prediciton:",y_pred)
        
        print(seconds_to_timedelta(y_pred))
        
    return training_data


In [17]:
# Model Training data sets
marathon_test_stop = date(2023,11,11)
marathon_test_stop_idx = (running_activities.loc[running_activities.index[(pd.to_datetime(running_activities["start_date_local"])).dt.date == marathon_test_stop]]).index[0]
fifteenK_stop = date(2023,11,22)
fifteenK_stop_idx = (running_activities.loc[running_activities.index[(pd.to_datetime(running_activities["start_date_local"])).dt.date == fifteenK_stop]]).index[0]

marathon_test_activities = running_activities[marathon_test_stop_idx : -1]
fifteenK_test_activities = running_activities[fifteenK_stop_idx : -1]

display(marathon_test_activities.head())
display(fifteenK_test_activities.head())

,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,workout_type,id,start_date,...,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp,race
11,2,Shake it out,6463.0,2128,2354,55.5,Run,0.0,10194282321,2023-11-10T14:52:33Z,...,0,False,38.0,42128847,1,a10194282321,s_ilFraruMYB?BIHa@JILCVT`@L`@h@hAb@x@Z`@`@G`@O...,2,NaN,False
12,2,Afternoon Run,8868.7,2791,3017,100.8,Run,NaN,10184937384,2023-11-08T19:29:36Z,...,0,False,54.0,42128847,1,a10184937384,aatwDrfrsQNRN[H@UPKROf@@F^TJBRR^b@^LLRF@FHHBHC...,2,NaN,False
13,2,Congress bats came out just to cheer my along ...,14664.8,4163,4896,138.5,Run,0.0,10179872931,2023-11-07T22:52:39Z,...,0,False,107.0,42128847,1,a10179872931,}}swDzfrsQfAz@FLWMmAQa@@k@RQXUJgDgAe@U_Bk@oCq@...,2,NaN,False
15,2,Ready for some givings w my friends 🦃,17733.1,4821,5456,141.3,Run,0.0,10167948372,2023-11-05T21:48:50Z,...,0,False,156.0,42128847,1,a10167948372,}~swDfgrsQg@Ne@ZCj@ORsIuCcJkCqFcAoRaFwAk@@GECs...,2,NaN,False
16,2,For the horns🤘🏼,8863.3,2593,2874,74.8,Run,0.0,10158800614,2023-11-04T13:07:49Z,...,0,False,48.0,42128847,1,a10158800614,iatwDjirsQGRWFO?aAi@s@UoBe@kAe@aBi@_@C]TQTQHOK...,2,NaN,False


,resource_state,name,distance,moving_time,elapsed_time,total_elevation_gain,sport_type,workout_type,id,start_date,...,total_photo_count,has_kudoed,suffer_score,athlete.id,athlete.resource_state,map.id,map.summary_polyline,map.resource_state,average_temp,race
2,2,Manager says log off early 🦃,8061.0,2699,2844,76.3,TrailRun,0.0,10264597941,2023-11-22T19:10:47Z,...,0,False,33.0,42128847,1,a10264597941,weebH`wxgV`An@dBj@q@o@F~@R~@n@D^MPb@j@h@VDFV\l...,2,NaN,False
3,2,Better than yesterday,9857.0,3392,3614,111.5,TrailRun,0.0,10258415903,2023-11-21T17:56:31Z,...,0,False,76.0,42128847,1,a10258415903,ghebHxyxgVBBXUbBe@TERDRnANd@R\z@S\l@PJFp@ZELLN...,2,NaN,False
4,2,Fighting a losing battle with my calf,6452.7,2521,2814,51.7,TrailRun,0.0,10252889734,2023-11-20T18:45:54Z,...,0,False,49.0,42128847,1,a10252889734,_hebHvxxgVxAYLGTEP@RHRNBXDx@Nb@ZDNEXQB@BT\\LNB...,2,NaN,False
5,2,A classic Washington welcome,12899.9,4366,5036,76.5,Run,0.0,10248228646,2023-11-19T19:31:37Z,...,1,False,91.0,42128847,1,a10248228646,whebH~yxgVWKcAA}Df@OOa@mAGy@WkAA}@DSVMzAXvAG`B...,2,NaN,False
6,2,Afternoon Run,8712.1,3163,3170,43.2,Run,NaN,10236722335,2023-11-17T23:36:21Z,...,0,False,14.0,42128847,1,a10236722335,q}swDpjrsQu@_@o@MW^a@KoAa@mA]]G_@Os@QcA_@eBe@[...,2,NaN,False


In [18]:
# Test models on upcoming races
richmond_marathon_input_dict = {
        "race_date": "2023-11-11T07:00:00Z",
        "start_latlng": [37.5407, -77.4360],
        "distance_miles": 26.2,
        "elev_gain_ft": 490,
        "elev_high_ft": 272
}

seattle_15k_input_dict = {
        "race_date": "2023-11-23T09:30:00Z",
        "start_latlng": [47.6798, -122.2536],
        "distance_miles": 9.32,
        "elev_gain_ft": 342,
        "elev_high_ft": 67
}

race_predictor(marathon_test_activities, richmond_marathon_input_dict, "linear_reg")
#race_predictor(fifteenK_test_activities, seattle_15k_input_dict, "linear_reg")

A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,start_date_local,start_latlng,distance,moving_time,total_elevation_gain,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,race,prediction
11,2023-11-10T09:52:33Z,"[38.88138284906745, -77.10249150171876]",6463.0,2128,55.5,4.041,84.2,145.8,165.0,84.7,False,False
12,2023-11-08T13:29:36Z,"[30.254413532093167, -97.74713435210288]",8868.7,2791,100.8,5.486,80.7,147.2,164.0,203.2,False,False
13,2023-11-07T16:52:39Z,"[30.253916485235095, -97.74717961438]",14664.8,4163,138.5,7.044,81.9,153.4,178.0,186.0,False,False
15,2023-11-05T15:48:50Z,"[30.2540785074234, -97.7472332585603]",17733.1,4821,141.3,8.456,82.9,159.0,180.0,187.4,False,False
16,2023-11-04T08:07:49Z,"[30.254457034170628, -97.74757917970419]",8863.3,2593,74.8,6.339,82.5,145.2,163.0,172.6,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
791,2021-08-26T18:04:04Z,"[30.301983, -97.698552]",9668.6,3068,40.8,4.100,77.4,140.8,161.0,189.5,False,False
792,2021-08-25T19:01:29Z,"[30.413111, -97.719967]",11280.3,3320,102.9,4.500,78.5,154.5,177.0,238.0,False,False
793,2021-08-24T19:06:32Z,"[30.274706, -97.764652]",8082.5,2307,58.6,5.600,80.9,145.4,183.0,152.5,False,False
794,2021-08-23T18:56:14Z,"[30.286558, -97.742625]",11385.5,3307,93.8,8.600,79.3,155.6,176.0,183.4,False,False


,start_date_local,start_latlng,distance,moving_time,total_elevation_gain,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,race,prediction
11,2023-11-10T09:52:33Z,"[38.88138284906745, -77.10249150171876]",6463.000,2128,55.500,4.0410,84.20,145.80,165.0,84.7000,False,False
12,2023-11-08T13:29:36Z,"[30.254413532093167, -97.74713435210288]",8868.700,2791,100.800,5.4860,80.70,147.20,164.0,203.2000,False,False
13,2023-11-07T16:52:39Z,"[30.253916485235095, -97.74717961438]",14664.800,4163,138.500,7.0440,81.90,153.40,178.0,186.0000,False,False
15,2023-11-05T15:48:50Z,"[30.2540785074234, -97.7472332585603]",17733.100,4821,141.300,8.4560,82.90,159.00,180.0,187.4000,False,False
16,2023-11-04T08:07:49Z,"[30.254457034170628, -97.74757917970419]",8863.300,2593,74.800,6.3390,82.50,145.20,163.0,172.6000,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
792,2021-08-25T19:01:29Z,"[30.413111, -97.719967]",11280.300,3320,102.900,4.5000,78.50,154.50,177.0,238.0000,False,False
793,2021-08-24T19:06:32Z,"[30.274706, -97.764652]",8082.500,2307,58.600,5.6000,80.90,145.40,183.0,152.5000,False,False
794,2021-08-23T18:56:14Z,"[30.286558, -97.742625]",11385.500,3307,93.800,8.6000,79.30,155.60,176.0,183.4000,False,False
795,2021-08-22T18:03:45Z,"[30.272434, -97.632629]",16105.000,4835,131.700,4.2000,78.80,152.70,174.0,197.4000,False,False


Retrieving list of weather stations...
Weather stations retrieved.


,name,country,region,wmo,icao,latitude,longitude,elevation,timezone,hourly_start,hourly_end,daily_start,daily_end,monthly_start,monthly_end,distance
id,,,,,,,,,,,,,,,,
72405,Washington National Airport,US,DC,72405,KDCA,38.8500,-77.0333,5.0,America/New_York,1936-09-01,2023-11-26,1936-09-01,2023-11-20,1936-01-01,2022-01-01,6669.082961
72254,Camp Mabry/Austin City Asos,US,TX,72254,KATT,30.3167,-97.7667,201.0,America/Chicago,2000-01-01,2023-11-26,1938-06-01,2023-11-19,1938-01-01,2022-01-01,7588.073798
KTKI0,McKinney,US,TX,<NA>,KTKI,33.1779,-96.5905,179.0,America/Chicago,2006-01-01,2023-11-26,2006-01-01,2022-04-25,2006-01-01,2022-01-01,7402.455335
72253,San Antonio International Airport,US,TX,72253,KSAT,29.5333,-98.4667,247.0,America/Chicago,1946-08-14,2023-11-26,1946-08-14,2023-11-20,1946-01-01,2022-01-01,10454.051693
KAPA0,Denver / Dove Valley,US,CO,<NA>,KAPA,39.5701,-104.8493,1794.0,America/Denver,2006-01-01,2023-11-26,2006-01-01,2022-04-24,2006-01-01,2022-01-01,12660.544227


...

,start_date_local,distance,moving_time,total_elevation_gain,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,race,prediction,start_time,start_lat,start_lng,avg_temp
11,2023-11-10,6463.0,2128,55.5,4.041,84.2,145.8,165.0,84.7,False,False,09:52:33,38.88,-77.10,11.9
12,2023-11-08,8868.7,2791,100.8,5.486,80.7,147.2,164.0,203.2,False,False,13:29:36,30.25,-97.75,22.9
13,2023-11-07,14664.8,4163,138.5,7.044,81.9,153.4,178.0,186.0,False,False,16:52:39,30.25,-97.75,22.8
15,2023-11-05,17733.1,4821,141.3,8.456,82.9,159.0,180.0,187.4,False,False,15:48:50,30.25,-97.75,19.6
16,2023-11-04,8863.3,2593,74.8,6.339,82.5,145.2,163.0,172.6,False,False,08:07:49,30.25,-97.75,19.0


How many nulls?
start_date_local        0
distance                0
moving_time             1
total_elevation_gain    0
max_speed               0
average_cadence         0
average_heartrate       0
max_heartrate           0
elev_high               0
race                    0
prediction              0
start_time              0
avg_temp                0
dtype: int64
HERE
TRAIONGON DTATA


,distance,moving_time,total_elevation_gain,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,race,prediction,avg_temp,month_sin,month_cos,day_sin,day_cos,sec_sin,sec_cos,2021,2022,2023
217,20935.900,5372,0.402734,0.643375,1.416905,1.370875,1.048872,0.402137,True,False,-0.607822,5.000000e-01,-0.866025,0.937752,0.347305,0.999976,-0.006909,False,False,True
298,21294.700,5384,0.442312,0.485005,0.782051,1.743401,0.754070,0.075154,True,False,-0.549350,8.660254e-01,0.500000,-0.651372,-0.758758,0.961181,-0.275917,False,False,True
333,21267.100,5192,0.126409,-0.151387,0.320339,2.048194,1.245406,0.094603,True,False,-1.157457,5.000000e-01,0.866025,-0.968077,-0.250653,0.921722,-0.387851,False,False,True
377,21294.800,5347,0.200768,1.330422,1.416905,1.382164,0.557536,0.086418,True,False,-0.794932,-2.449294e-16,1.000000,0.724793,0.688967,0.961081,-0.276266,False,True,False
386,5113.700,1158,0.028784,0.827364,2.975184,0.061393,-0.032068,0.051539,True,False,-0.853404,-5.000000e-01,0.866025,-0.988468,0.151428,0.788414,-0.615146,False,True,False
0,42164.708,None,0.358244,0.626956,1.382277,1.321205,0.714763,0.029557,True,True,-1.414734,-5.000000e-01,0.866025,0.790776,-0.612106,0.965926,-0.258819,False,False,True


coefficient of determination: 0.9817842520540216
intercept: 9.22991350368784
coefficients: [ 2.87024920e-01  6.00248908e+02 -2.28501728e+00 -9.77874464e+01
 -9.70207034e+01  1.44148183e+00  1.06982800e+02 -5.95220686e+02
 -5.32907052e-15  1.17498791e+01  2.04690418e+01 -4.87571617e+01
 -8.92637169e+00  5.16298538e+01 -4.27034821e+01]
prediciton: [11443.83743372]
3:10:43


,distance,moving_time,total_elevation_gain,max_speed,average_cadence,average_heartrate,max_heartrate,elev_high,race,prediction,avg_temp,month_sin,month_cos,day_sin,day_cos,sec_sin,sec_cos,2021,2022,2023
11,6463.000,2128,0.133125,-0.772059,2.802042,-0.401442,-0.621671,0.030206,False,False,-0.970347,-0.500000,0.866025,0.897805,-0.440394,0.527883,-0.849317,False,False,True
12,8868.700,2791,0.241785,0.069283,0.782051,-0.243401,-0.719938,0.073126,False,False,0.316033,-0.500000,0.866025,0.998717,-0.050649,-0.381070,-0.924546,False,False,True
13,14664.800,4163,0.332214,0.976418,1.474619,0.456495,0.655803,0.066896,False,False,0.304339,-0.500000,0.866025,0.988468,0.151428,-0.957130,-0.289658,False,False,True
15,17733.100,4821,0.338930,1.798546,2.051760,1.088659,0.852337,0.067403,False,False,-0.069881,-0.500000,0.866025,0.848644,0.528964,-0.840645,-0.541586,False,False,True
16,8863.300,2593,0.179420,0.565936,1.820904,-0.469173,-0.818205,0.062043,False,False,-0.140047,-0.500000,0.866025,0.724793,0.688967,0.848472,-0.529241,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
792,11280.300,3320,0.246822,-0.504809,-0.487658,0.580670,0.557536,0.085730,False,False,1.204805,-0.866025,-0.500000,-0.937752,0.347305,-0.964230,0.265065,True,False,False
793,8082.500,2307,0.140561,0.135659,0.897479,-0.446596,1.147139,0.054763,False,False,1.181417,-0.866025,-0.500000,-0.988468,0.151428,-0.958156,0.286246,True,False,False
794,11385.500,3307,0.224994,1.882389,-0.025946,0.704846,0.459268,0.065954,False,False,1.099556,-0.866025,-0.500000,-0.998717,-0.050649,-0.970049,0.242910,True,False,False
795,16105.000,4835,0.315903,-0.679482,-0.314516,0.377475,0.262734,0.071025,False,False,0.889057,-0.866025,-0.500000,-0.968077,-0.250653,-0.999866,0.016362,True,False,False
